In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

# Chapitre 4: Systèmes à plusieurs qubits

A présent, étudions un système à deux qubits. Il s'agit du système minimal pour étudier l'un des plus fascinants phénomènes prédit par la mécanique quantique: l'intrication quantique. Celle-ci est à la base de l'accélération dans les calculs de certains problèmes et est cruciale en information et en cryptographie quantiques.

Afin d'étudier des systèmes à plusieurs qubits, introduisons tout d'abord les notations utiles.

## 4.1: Représentation tensorielle des états quantiques et matrice densité

Comme nous l'avons vu dans les chapitres précédents, l'état d'un qubit est représenté par un ket $\ket \psi_1^{} = \alpha_1^{} \ket 0 + \beta_1^{} \ket{1}$. De même, l'état d'un second qubit se notera $\ket \phi_2^{} = \alpha_2^{} \ket 0 + \beta_2^{} \ket{1}$. L'état le plus général d'un système composé de deux qubits se note quant à lui

$\qquad \ket{\Psi_2} = a_{00}^{} \ket 0_2^{} \otimes \ket 0_1^{} + a_{01}^{} \ket 0_2^{} \otimes \ket 1_1^{} + a_{10}^{} \ket 1_2^{} \otimes \ket 0_1^{} + a_{11}^{} \ket 1_2^{} \otimes \ket 1_1^{}$,

où le symbole $\otimes$ est appelé *produit tensoriel*. On remarque l'ajout des indices "1" et "2" aux kets $\ket 0$ et $\ket 1$ afin de notifier à quel qubit ils correspondent. Bien que claire, la notation tensorielle et avec les indices peut s'avérer lourde. Lorsqu'il est aisé de retracer quelle composante appartient à quel qubit, on peut s'en passer et même encore plus compacter les notations en fusionnat les kets: $\ket{0}_2^{} \otimes \ket{0}_1^{} = \ket{00}$. Ainsi,

$\qquad \ket{\Psi_2} = a_{00}^{} \ket{00} + a_{01}^{} \ket{01} + a_{10}^{} \ket{10} + a_{11}^{} \ket{11}$. 

Plus tard, nous utiliserons cette noation mais pour le moment, gardons la notation explicite.

Il est intéressant de s'arrêter à quelques exemples d'états particuliers directement drivés de cet état général. Prenons pour commencer l'état

$\qquad \ket{\Psi_2} = a_{00}^{} \ket 0_2^{} \otimes \ket 0_1^{} + a_{01}^{} \ket 0_2^{} \otimes \ket 1_1^{}$. 

Sachant que le symbole $\otimes$ représente un produit, il est possible de mettre en évidence ce qui est commun aux deux termes, c'est-à-dire $\ket 0_2^{}$. Ceci conduit à

$\qquad \Psi_2^{} = \ket{0}_2^{} \otimes (a_{00}^{} \ket 0_1^{} + a_{01}^{} \ket 1_1^{})$.

Ceci traduit que le qubit 2 est dans l'état 0 et que le qubit 1 est dans un état superposé.



### Etats séparables

Certains états d'un système à deux qubits peuvent se récrire comme suit:

$\qquad \ket{\Psi_2^{}} =  \ket \phi_2^{} \otimes \ket \psi_1^{}$,

$\quad \ket \phi_2^{} \otimes \ket \psi_1^{} = \left(\alpha_2^{} \ket 0 + \beta_2^{} \ket{1}\right) \otimes \left(\alpha_1^{} \ket 0 + \beta_1^{} \ket{1}\right)$

$\qquad \qquad \qquad = \alpha_2^{} \alpha_1^{} \ket 0 \otimes \ket 0 + \alpha_2^{} \beta_1^{} \ket 0 \otimes \ket 1 + \beta_2^{} \alpha_1^{} \ket 1 \otimes \ket 0 + \beta_2^{} \beta_1^{} \ket 1 \otimes \ket 1$.

Parfois, on rajoute des indices aux kets $\ket 0$ et $\ket 1$ afin de notifier à quel qubit ils correspondent: $\ket 0_2^{} \otimes \ket 0_1^{}$. Bien que claire , cette notation peut s'avérer lourde. Lorsqu'il est aisé de retracer quelle composante appartient à quel qubit, on peut s'en passer. On peut même encore plus compacter les notations en fusionnat les kets: $\ket 0 \otimes \ket 0 = \ket{00}$. Ainsi, l'état $\ket \phi_2^{} \otimes \ket \psi_1^{}$ peut se récrire comme suit:

$\ket \phi_2^{} \otimes \ket \psi_1^{} = \alpha_2^{} \alpha_1^{} \ket{00} + \alpha_2^{} \beta_1^{} \ket{01} + \beta_2^{} \alpha_1^{} \ket{10} + \beta_2^{} \beta_1^{} \ket{11}.$

On comprend que le nombre d'états de base s'élève à 4, soit deux par qubit. Ainsi, avec 3 qubits, le nombre d'états s'élève à 8, et pour $N$ qubits, à $2^N_{}$. Dès lors, un état à $2$ qubits s'écrit avec des vecteurs à $2^2_{}$ composantes:

$\begin{pmatrix} 
\alpha_2^{} \alpha_1^{} \\
\alpha_2^{} \beta_1^{} \\
\beta_2^{} \alpha_1^{}\\
\beta_2^{} \beta_1^{} \\
\end{pmatrix}$

$~$

## 4.2: Opérations sur plusieurs qubits

Les portes quantiques que nous avons appliquées sur un seul qubit peuvent s'appliquer séparément sur plusieurs qubits à la fois, comme montré grâce au circuit ci-dessous. En se souvent que l'action d'une porte logique sur un qubit correspond à une multiplication par une matrice (unitaire) sur le vecteur représentatn le qubit, il suffit d'appliquer une matrice sur le premier qubit et séparément une deuxième matrice sur le deuxième qubit:

$\qquad (M_2^{} \ket \psi_2^{}) \otimes (M_1^{} \ket \phi_1^{}) =  M_2^{} \otimes M_1^{} (\ket \psi_2^{} \otimes \ket \phi_1^{})$.

Prenons l'exemple de deux portes de Hadamard appliquées chacune sur un qubit:

In [ ]:
# 1: Créer un objet "circuit à 1 qubit":
qc = QuantumCircuit(2)

# 2: Definir l'état initial:
etat_initial_1 = [1, 0]  # l'état |0>
qc.initialize(etat_initial_1, 0)
etat_initial_2 = [0, 1]  # l'état |0>
qc.initialize(etat_initial_2, 1)

# 3: Ajouter la porte de Hadamard sur chaque qubit:
qc.h(0)
qc.h(1)

# 4: Appliquer le circuit sur l'état initial:
etat_final = Statevector.from_instruction(qc)

print("Etat du système à l'issu du circuit:", etat_final)

# Schéma du circuit
qc.draw('mpl')

Après que les qubits dans les états $\ket 0$ et $\ket 1$ soient passés chacun de leur côté par une porte de Hadamard, l'état final de l'ensemble des deux qubits s'écrit:

$\qquad (H \ket 1) \otimes (H\ket{0}) = (\frac{1}{\sqrt{2}} \ket 0 - \frac{1}{\sqrt{2}} \ket 1) \otimes (\frac{1}{\sqrt{2}} \ket 0 + \frac{1}{\sqrt{2}} \ket 1) = \frac{1}{2} \ket{00} + \frac{1}{2} \ket{01} - \frac{1}{2} \ket{10} - \frac{1}{2} \ket{11} $.

Il existe également des portes quantiques qui agissent nécessairement sur deux qubits à la fois (les matrices correspondantes ne peuvent alors pas s'écrire comme un seul produit tensoriel de deux matrices, mais bien comme une somme de produits tensoriels de deux matrices). Ce type de porte a pour effet d'*intriquer* les qubits (*entangle* en anglais). En voici quelques exemples:

Portes **CNOT**: $C_X^{} ...$

Portes **SWAP**:

$~$

## 4.3: Mesures et mesures partielles

## 4.4: Intrication et états de Bell

In [ ]:
# 1: Créer un objet "circuit à 1 qubit":
qc = QuantumCircuit(2)

# 2: Definir l'état initial:
psi_0 = [1, 0] # le qubit 0 est dans l'état |0>
psi_1 = [1, 0] # le qubit 1 est dans l'état |1>
qc.initialize(psi_0, 0)
qc.initialize(psi_1, 1)

# 3: Ajouter la porte de Hadamard au circuit:
qc.h(0)
qc.cx(0,1)

# 4: Appliquer le circuit sur l'état initial:
etat_final = Statevector(qc)

print("Etat du système à l'issue du circuit:")
display(etat_final.draw('latex'))

# Schéma du circuit
qc.draw('mpl')

$~$

## 4.5: Circuit à deux qubits

**Pas très intuitif comme example: pourquoi pas juste le donner avec la contruction d'une paire de Bell?**

**De plus, je ne pense pas qu'il soit très conventionnel d'initialiser un circuit ainsi.**

En se rappelant des commandes utilisées dans le chapitre 3 pour construire des circuits à 1 qubit et y ajouter des portes logiques, adaptons-les pour construire des circuits à 2 qubits et y insérer des portes logiques à 1 qubits et à 2 qubits.

In [ ]:
# 1: Créer un objet "circuit à 1 qubit":
qc = QuantumCircuit(2)

# 2: Definir l'état initial:
psi_0 = [1, 0] # le qubit 0 est dans l'état |0>
psi_1 = [0, 1] # le qubit 1 est dans l'état |1>
qc.initialize(psi_0, 0)
qc.initialize(psi_1, 1)

# 3: Ajouter la porte de Hadamard au circuit:
qc.h(0)
qc.x(1)
qc.cx(0,1)

# 4: Appliquer le circuit sur l'état initial:
etat_final = Statevector(qc)

print("Etat du système à l'issue du circuit:")
display(etat_final.draw('latex'))

# Schéma du circuit
qc.draw('mpl')
